## Cálculo da emissão e distância percorrida 

Nesse notebook é calculada a distância percorrida entre os pontos e a emissão de poluentes

In [1]:
import pandas as pd
import random
from utils.save_csv import save_csv
from utils.load_csv import load_csv

In [2]:
consumo_diesel = load_csv('consumo_diesel.csv')

veiculos = load_csv('veiculos.csv')

posicoes_df = load_csv('posicoes_onibus.csv')

In [3]:
consumo_diesel['emissa_no2_l_diesel'] = [random.uniform(0.02, 0.05) for _ in range(len(consumo_diesel))]
consumo_diesel['emissa_co2_l_diesel'] = [random.uniform(0.05, 0.1) for _ in range(len(consumo_diesel))]

In [4]:
consumo_diesel.head()

,tecnologia,sem_ar_l_km,sem_ar_kg_km,com_ar_l_km,com_ar_kg_km,emissa_no2_l_diesel,emissa_co2_l_diesel
0,Miniônibus,0.30,0.252,0.35,0.294,0.045362,0.077357
1,Midiônibus,0.40,0.336,0.47,0.395,0.022576,0.087792
2,Básico,0.46,0.386,0.53,0.445,0.033502,0.066964
3,Padron,0.55,0.462,0.63,0.529,0.046676,0.053169
4,Articulado (18m),0.71,0.596,0.80,0.672,0.031881,0.086454


In [5]:
def get_tipo_onibus(id_onibus):

    onibus = veiculos[veiculos['id_veiculo']==id_onibus]

    return onibus['modelo'].unique()[0]

In [6]:
def get_fator_consumo(tipo_veiculo):

    cosumo_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]

    return cosumo_modelo['com_ar_l_km'].unique()[0]

def get_consumo_co2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_co2_modelo = fator_emissao_modelo['emissa_co2_l_diesel'].unique()[0]
    emissao_co2 = fator_emissao_co2_modelo * consumo_fato

    return emissao_co2

def get_consumo_no2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_no2_modelo = fator_emissao_modelo['emissa_no2_l_diesel'].unique()[0]
    emissao_no2 = fator_emissao_no2_modelo * consumo_fato

    return emissao_no2

In [ ]:
data_frame = []

for id_onibus in posicoes_df['id_onibus'].unique():
    df_onibus = posicoes_df[posicoes_df['id_onibus'] == id_onibus].reset_index(drop=True)
    df_onibus.sort_values(by='t', inplace=True)
    for i, row in df_onibus.iterrows():
 
        if i < len(df_onibus)-1:
            x_atual = row['x']
            x_depois = df_onibus.loc[i+1, 'x']
            y_atual = row['y']
            y_depois = df_onibus.loc[i+1, 'y']
            momento_inicial = row['t']
            momento_final = df_onibus.loc[i+1, 't']
            tipo_veiculo = get_tipo_onibus(row['id_onibus'])
            litro_diesel = get_fator_consumo(tipo_veiculo)
            
            #mas tem que reprojetar para SIRGAS 2000
            dist = (x_depois - x_atual)**2 + (y_depois - y_atual)**2
            
            dados = {
                'id_onibus' : row['id_onibus'],
                'distancia_percorrida' : dist,
                'ponto_inicial' : [x_atual, y_atual],
                'ponto_final' : [x_depois, y_depois],
                'momento_inicial' : momento_inicial,
                'momento_final' : momento_final,
                'modelo' : tipo_veiculo,
                'litro_diesel_km' : litro_diesel,
                'emissao_no2' : get_consumo_no2(tipo_veiculo, dist),
                'emissao_co2' : get_consumo_co2(tipo_veiculo, dist)
            }

            data_frame.append(dados)

df_final = pd.DataFrame(data_frame)

In [8]:
df_final = df_final.merge(
    veiculos[['id_veiculo', 'eletrico']],
    left_on='id_onibus',
    right_on='id_veiculo',
    how='left').drop(columns='id_veiculo')

In [9]:
df_final

,id_onibus,distancia_percorrida,ponto_inicial,ponto_final,momento_inicial,momento_final,modelo,litro_disel_km,emissao_no2,emissao_co2,eletrico
0,1001,0.004639,"[-46.562178411294376, -23.59317831488312]","[-46.61476116660492, -23.549888807753703]",2025-08-13T11:03:36.770380,2025-08-13T11:29:36.770380,Miniônibus,0.35,0.000074,0.000126,True
1,1001,0.000620,"[-46.5746826521639, -23.54392364318316]","[-46.57147515362678, -23.519225956077552]",2025-08-13T11:09:36.770380,2025-08-13T11:48:36.770380,Miniônibus,0.35,0.000010,0.000017,True
2,1001,0.003623,"[-46.61476116660492, -23.549888807753703]","[-46.5607253850038, -23.57640262532844]",2025-08-13T11:29:36.770380,2025-08-13T11:43:36.770380,Miniônibus,0.35,0.000058,0.000098,True
3,1001,0.000402,"[-46.59429954744935, -23.53972533591911]","[-46.5746826521639, -23.54392364318316]",2025-08-13T11:43:36.770380,2025-08-13T11:09:36.770380,Miniônibus,0.35,0.000006,0.000011,True
4,1001,0.005555,"[-46.57147515362678, -23.519225956077552]","[-46.562178411294376, -23.59317831488312]",2025-08-13T11:48:36.770380,2025-08-13T11:03:36.770380,Miniônibus,0.35,0.000088,0.000150,True
5,1002,0.003098,"[-46.567732536142, -23.561769728930745]","[-46.62332140193004, -23.55898736327238]",2025-08-13T11:02:36.770380,2025-08-13T11:19:36.770380,Básico,0.53,0.000055,0.000110,False
6,1002,0.001501,"[-46.573339020310016, -23.55972354581055]","[-46.58651877163993, -23.523290333156798]",2025-08-13T11:02:36.770380,2025-08-13T11:05:36.770380,Básico,0.53,0.000027,0.000053,False
7,1002,0.001602,"[-46.58651877163993, -23.523290333156798]","[-46.60407945016048, -23.55926150257764]",2025-08-13T11:05:36.770380,2025-08-13T10:55:36.770380,Básico,0.53,0.000028,0.000057,False
8,1002,0.000108,"[-46.62332140193004, -23.55898736327238]","[-46.62564580880299, -23.548843607255456]",2025-08-13T11:19:36.770380,2025-08-13T11:32:36.770380,Básico,0.53,0.000002,0.000004,False
9,1002,0.002854,"[-46.62564580880299, -23.548843607255456]","[-46.573339020310016, -23.55972354581055]",2025-08-13T11:32:36.770380,2025-08-13T11:02:36.770380,Básico,0.53,0.000051,0.000101,False


In [10]:
save_csv(df_final, "df_final.csv")

Base salva em data\df_final.csv
